In [1]:
from lavis.models import model_zoo

import re
import torch
from torch import nn
from torchvision import transforms

import json

from PIL import Image

import cv2
import numpy as np

from skimage import transform as skimage_transform
from scipy.ndimage import filters
from matplotlib import pyplot as pl

import requests
from lavis.models import load_model_and_preprocess, load_model

from tqdm import tqdm

from CustomDataset import CustomDataset
from torch.utils.data import DataLoader


# model optim
from torch.optim import AdamW, SGD

# lr schedulers
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

/opt/conda/envs/pytorch/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model, vision_processer, lang_processer = load_model_and_preprocess(name="blip2", model_type="pretrain")

In [4]:
model.to(device)

Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [5]:
dataset = CustomDataset("/home/ubuntu/Text_Guided_Image_Editing/data")

dataset built with 18 instances


In [6]:
print(dataset[0])

(<PIL.Image.Image image mode=RGB size=781x781 at 0x7FBB3C20E9E0>, 'dog', 'a brown-and-white dog ')


In [7]:
train_dataloader = DataLoader(dataset, shuffle=True, batch_size=1)

In [8]:
epoch=20
lr=1e-5
eps=5e-5

In [9]:
model.optimizer = AdamW(model.parameters(), lr=lr, eps=eps)

train_steps = epoch*len(train_dataloader)
warm_step = int(train_steps*0.1)
model.scheduler = get_cosine_schedule_with_warmup(model.optimizer, num_warmup_steps=warm_step,num_training_steps=train_steps)

In [11]:
for e in tqdm(range(epoch)):
    
    losses = 0
    for image, label, text in dataset:
        
        image = vision_processer["eval"](image).unsqueeze(0)
        
        samples = {'image': image.to(device), 'text_input': text}
        
        output = model(samples)
        loss = output['loss']
        
        losses += loss.item()
        
        model.zero_grad()
        loss.backward()
        
        model.optimizer.step()  # backprop to update the weights

        if model.scheduler is not None:
            model.scheduler.step()  # update learning rate schedule 
    
    print("epoch: ", epoch, "losses: ", losses/len(dataset))
        

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]

epoch:  0


 20%|█████████████████                                                                    | 1/5 [00:02<00:11,  2.83s/it]

losses:  1.857012563281589
epoch:  1


 40%|██████████████████████████████████                                                   | 2/5 [00:05<00:08,  2.84s/it]

...

losses:  1.7635159757402208
epoch:  3


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [00:11<00:02,  2.85s/it]

losses:  1.652252389325036
epoch:  4


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.85s/it]

losses:  1.558305905924903
